In [1]:
!pip install plyfile

In [ ]:
!unzip PointCNN_Pytorch.zip

In [3]:
from __future__ import print_function
import argparse
import os
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from data_class import ModelNet40,ShapeNetPart
from transforms_3d import *


from pointcnn import PointCNNCls,PointCNN_partseg

from utils import progress_bar, adjust_lr_steep, log_row

In [4]:
def calculate_shape_IoU(pred_np, seg_np, label, class_choice):
    
    shape_ious = []
    
    for shape_idx in range(seg_np.shape[0]):
        if not class_choice:
            # print (label[shape_idx][0])
            idx = label[shape_idx][0]
            start_index = index_start[idx]
            num = seg_num[idx]
            parts = range(start_index, start_index + num)
        else:
            parts = range(seg_num[label[0]])
        part_ious = []
        for part in parts:
            I = np.sum(np.logical_and(pred_np[shape_idx] == part, seg_np[shape_idx] == part))
            U = np.sum(np.logical_or(pred_np[shape_idx] == part, seg_np[shape_idx] == part))
            if U == 0:
                iou = 1  # If the union of groundtruth and prediction points is empty, then count part IoU as 1
            else:
                iou = I / float(U)
            # print ('iou ', part, iou)
            part_ious.append(iou)
        shape_ious.append(np.mean(part_ious))
    

    return shape_ious



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PointCNN_partseg(50)
model = model.to(device) 
model = nn.DataParallel(model) 

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

data_,model_,name_="ShapeNet", "PointCNN", "seg"

In [6]:
def save_ckpt(epoch, model, optimizer, acc_list):
    
    if not os.path.isdir('checkpoints'):
        os.mkdir('checkpoints')
    
    if not os.path.isdir('checkpoints/%s_%s_%s'%(data_,model_,name_)):
        os.mkdir('checkpoints/%s_%s_%s'%(data_,model_,name_))
    if (epoch % 20 == 0) or (epoch in adj_lr['steps']) or (acc_list[-1] > max(acc_list[:-1])):
        print('=====> Saving checkpoint...')
        state = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'acc_list': acc_list,
                }
        torch.save(state, 'checkpoints/%s_%s_%s/epoch_%d.pth' % (data_,model_,name_, epoch))
        print('Successfully save checkpoint at epoch %d' % epoch)


In [7]:
adj_lr = {'steps' : [int(temp) for temp in [50, 80, 120, 150]], 'decay_rates' : [float(temp) for temp in [0.1, 0.1, 0.2, 0.2]]}

seg_num = [4, 2, 2, 4, 4, 3, 3, 2, 4, 2, 6, 2, 3, 3, 3, 3]
index_start = [0, 4, 6, 8, 12, 16, 19, 22, 24, 28, 30, 36, 38, 41, 44, 47]
seg_num_all = 50
seg_start_index = 0

train_tfs = compose([rotate_y(), 
                             rand_scale(), 
                             rand_translate(), 
                             jitter(), 
                             normalize()
                            ])
test_tfs = normalize()

batch_size = 16

train_loader = DataLoader(ShapeNetPart(partition='trainval', num_points=2048, class_choice=None, transforms=train_tfs), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(ShapeNetPart(partition='test', num_points=2048, class_choice=None, transforms=test_tfs), batch_size=batch_size, shuffle=False)

In [38]:
def test(model, test_loader):
    model.eval()
    test_true_cls = []
    test_pred_cls = []
    test_true_seg = []
    test_pred_seg = []
    test_label_seg = []
    correct = 0
    total = 0
    for j, data in enumerate(test_loader, 0):

        points, label, seg = data
        points, seg  = points.to(device), seg.to(device)          
        points = points.transpose(2, 1) # to be shape batch_size*3*N     
        seg_pred = model(points)
        seg_pred = seg_pred.permute(0, 2, 1)          
        loss=F.cross_entropy(seg_pred.reshape(-1, seg_num_all), seg.view(-1))
            
        pred_choice = seg_pred.data.max(2)[1]

        correct += pred_choice.eq(seg.data).cpu().sum()
        total += pred_choice.size(0)* pred_choice.size(1)

        seg_np = seg.cpu().numpy()
        pred_np = pred_choice.detach().cpu().numpy()
        label = label.numpy() # added 

        test_true_cls.append(seg_np.reshape(-1))
        test_pred_cls.append(pred_np.reshape(-1))

        test_true_seg.append(seg_np)
        test_pred_seg.append(pred_np)

        test_label_seg.append(label.reshape(-1, 1))
        pbar.set_postfix({'step':i+1,'Test Loss:' : total_loss/(i+1),'Test Acc':100.*correct.item()/total})
       

    test_true_cls = np.concatenate(test_true_cls)
    test_pred_cls = np.concatenate(test_pred_cls)

    test_acc = metrics.accuracy_score(test_true_cls, test_pred_cls)
    avg_per_class_acc = metrics.balanced_accuracy_score(test_true_cls, test_pred_cls)

    test_true_seg = np.concatenate(test_true_seg, axis=0)
    test_pred_seg = np.concatenate(test_pred_seg, axis=0)
    test_label_seg = np.concatenate(test_label_seg)

    test_ious = calculate_shape_IoU(test_pred_seg, test_true_seg, test_label_seg, None)
    outstr = 'Test %d, loss: %.6f, test acc: %.6f, test avg acc: %.6f, test iou: %.6f' % (epoch,
                                                                                              total_loss/(j+1),
                                                                                              100*(correct/total),
                                                                                              avg_per_class_acc,
                                                                                              np.mean(test_ious))
        
    print(outstr)
    return total_loss/(j+1), 100.*correct.item()/total

In [35]:
if not os.path.isdir('logs_train'):
        os.mkdir('logs_train')
logname = ('logs_train/%s_%s_%s.csv' % (data_,model_,name_))    
    
if os.path.exists(logname):
        with open(logname, 'a') as logfile:
            log_row(logname, [''])
            log_row(logname, [''])

acc_list=[0]

In [37]:
from tqdm.notebook import tqdm
import sklearn.metrics as metrics

for epoch in range(200):
        model.train()
        print('\nEpoch: %d' % epoch)
        optimizer.param_groups = adjust_lr_steep(0.001, optimizer.param_groups, epoch, adj_lr)
        train_true_cls = []
        train_pred_cls = []
        train_true_seg = []
        train_pred_seg = []
        train_label_seg = []
        correct = 0
        total = 0
        total_loss=0
        pbar=tqdm(train_loader)
        for i, data in  enumerate(pbar): #enumerate(train_loader, 0):
            points, label, seg = data
            points, seg  = points.to(device), seg.to(device)
            
            points = points.transpose(2, 1) # to be shape batch_size*3*N

            optimizer.zero_grad()


            seg_pred = model(points)
            seg_pred = seg_pred.permute(0, 2, 1)
            
            loss =  F.cross_entropy(seg_pred.reshape(-1, seg_num_all), seg.view(-1))
           #print(loss.item())
            
            loss.backward()
            optimizer.step()
            pred_choice = seg_pred.data.max(2)[1]
            total_loss+=loss.item()
            
            correct += pred_choice.eq(seg.data).cpu().sum()
            total += pred_choice.size(0)* pred_choice.size(1)

            seg_np = seg.cpu().numpy()                
            pred_np = pred_choice.detach().cpu().numpy()
            label = label.numpy() # added 
            temp_label = label.reshape(-1, 1)

            train_true_cls.append(seg_np.reshape(-1))       
            train_pred_cls.append(pred_np.reshape(-1))      

            train_true_seg.append(seg_np)
            train_pred_seg.append(pred_np)

            train_label_seg.append(temp_label)
           
        
 
            pbar.set_postfix({'step':i+1,'Train Loss:' : total_loss/(i+1),'Train Acc':100.*correct.item()/total})
            

        train_loss, train_acc = total_loss/(i+1), 100.*correct.item()/total 

        train_true_cls = np.concatenate(train_true_cls)
        train_pred_cls = np.concatenate(train_pred_cls)

        train_acc = metrics.accuracy_score(train_true_cls, train_pred_cls)
        avg_per_class_acc = metrics.balanced_accuracy_score(train_true_cls.data, train_pred_cls.data)

        train_true_seg = np.concatenate(train_true_seg, axis=0)
        train_pred_seg = np.concatenate(train_pred_seg, axis=0)
        train_label_seg = np.concatenate(train_label_seg, axis=0)

        train_ious = calculate_shape_IoU(train_pred_seg, train_true_seg, train_label_seg, None)
        
        outstr = 'Train %d, loss: %.6f, train acc: %.6f, train avg acc: %.6f, train iou: %.6f' % (epoch, train_loss/(i+1), train_acc, avg_per_class_acc, np.mean(train_ious))
        
        ### Test in batch 
        test_loss, test_acc = test(model, test_loader)        
        acc_list.append(test_acc)  

        ### Keep tracing
        log_row(logname, [epoch, train_loss, train_acc, test_loss, test_acc, 
                optimizer.param_groups[0]['lr'], max(acc_list), np.argmax(acc_list)-1])
        save_ckpt( epoch, model, optimizer, acc_list)
       
        
            


Epoch: 0


KeyboardInterrupt: ignored

In [25]:
seg_np.shape

(16, 2048)

In [22]:
seg_pred.cpu().numpy()

RuntimeError: ignored

In [16]:
train_pred_cls

[tensor([0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.4047, 0.7964], device='cuda:0')]